In [64]:
# Step 0. Import libraries and custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
# Step 1. Load data, show info and sample
# 1.1 Read data
df_raw = pd.read_excel(
   'https://github.com/datagy/mediumdata/raw/master/sample_pivot.xlsx', 
   parse_dates=['Date']
)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1000 non-null   datetime64[ns]
 1   Region  1000 non-null   object        
 2   Type    1000 non-null   object        
 3   Units   911 non-null    float64       
 4   Sales   1000 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 39.2+ KB


In [66]:
# 1.2 Show sample
df_raw.sample(5, random_state=1234)

,Date,Region,Type,Units,Sales
681,2020-11-27,East,Children's Clothing,32.0,342
990,2020-12-17,North,Children's Clothing,7.0,217
155,2020-06-13,East,Children's Clothing,30.0,336
768,2020-06-25,North,Women's Clothing,5.0,644
438,2020-10-07,North,Women's Clothing,30.0,462


In [67]:
# Step 2. Transform to standard format
# 2.1 Create a pandas chain to convert to final form
df = (
    df_raw
    .copy()
    .set_axis(
        df_raw
        .columns.str.lower()
        .str.replace(' ','_')
        .str.replace(r'\W','',regex=True)
        .str.slice(0,40), axis=1
    )
    .astype({'region':'category', 
             'type':'category'})
    .assign(
        date = lambda x: pd.to_datetime(x['date'], format='%Y-%m-%d')
    )
    .set_index('date')
)

In [68]:
# Step 3. Perform busines analysis
# 3.1 Get the summary, for categorical and numerical columns


#categorical_summary = df.describe(include=['object'])
#display(categorical_summary)

numerical_summary = df.describe()
display(numerical_summary)


,units,sales
count,911.000000,1000.000000
mean,19.638858,427.254000
std,9.471309,253.441362
min,3.000000,33.000000
25%,12.000000,224.000000
50%,20.000000,380.000000
75%,28.000000,575.000000
max,35.000000,1155.000000


In [69]:
# 3.2 Get the sum of total sales
total_sales = df['sales'].sum()
print(total_sales)

427254


In [70]:
# 3.3 Get the average of sales
average_sales = df['sales'].mean()
print(average_sales)

427.254


In [71]:
# 3.4 Get the sum of sales by region
sales_by_region = df.groupby('region')['sales'].sum()
print(sales_by_region)

region
East     167763
North    138700
South     59315
West      61476
Name: sales, dtype: int64


/tmp/ipykernel_7989/4191440647.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales_by_region = df.groupby('region')['sales'].sum()


In [81]:
# 3.4 Get the average sales by region
average_sales_by_region = df.groupby('region')['sales'].mean()
print(average_sales_by_region)

region
East     408.182482
North    438.924051
South    432.956204
West     452.029412
Name: sales, dtype: float64


/tmp/ipykernel_7989/2957225678.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_sales_by_region = df.groupby('region')['sales'].mean()


In [83]:
# 3.5 Get the sum, mean and count of sales by region
sales_summary_by_region = df.groupby('region')['sales'].agg(['sum', 'mean', 'count'])
print(sales_summary_by_region)

           sum        mean  count
region                           
East    167763  408.182482    411
North   138700  438.924051    316
South    59315  432.956204    137
West     61476  452.029412    136


/tmp/ipykernel_7989/2004636370.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales_summary_by_region = df.groupby('region')['sales'].agg(['sum', 'mean', 'count'])


In [86]:
# 3.6 Create a pivot table that gets total units by type
pivot_table_units_by_type = df.pivot_table(values='units', index='type', aggfunc='sum')
print(pivot_table_units_by_type)

                      units
type                       
Children's Clothing  5887.0
Men's Clothing       3974.0
Women's Clothing     8030.0


/tmp/ipykernel_7989/4204821016.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table_units_by_type = df.pivot_table(values='units', index='type', aggfunc='sum')


In [88]:
# 3.7 Display the dataset showing only the East region
east_region_data = df[df['region'] == 'East']
print(east_region_data)

           region                 type  units  sales
date                                                
2020-07-11   East  Children's Clothing   18.0    306
2020-02-28   East  Children's Clothing   26.0    832
2020-03-25   East     Women's Clothing   29.0    609
2020-11-03   East  Children's Clothing   34.0    374
2020-05-01   East       Men's Clothing   10.0    140
...           ...                  ...    ...    ...
2020-03-20   East       Men's Clothing   14.0    238
2020-06-05   East  Children's Clothing   11.0    220
2020-05-04   East       Men's Clothing    6.0    108
2020-10-18   East     Women's Clothing   19.0    399
2020-02-11   East  Children's Clothing   35.0    735

[411 rows x 4 columns]


In [91]:
# 3.8 Display the dataset showing only the East and West regions
east_west_region_data = df[df['region'].isin(['East', 'West'])]
print(east_west_region_data)

           region                 type  units  sales
date                                                
2020-07-11   East  Children's Clothing   18.0    306
2020-02-28   East  Children's Clothing   26.0    832
2020-03-19   West     Women's Clothing    3.0     33
2020-03-25   East     Women's Clothing   29.0    609
2020-11-03   East  Children's Clothing   34.0    374
...           ...                  ...    ...    ...
2020-02-08   West       Men's Clothing   32.0    928
2020-05-04   East       Men's Clothing    6.0    108
2020-11-17   West       Men's Clothing   27.0    486
2020-10-18   East     Women's Clothing   19.0    399
2020-02-11   East  Children's Clothing   35.0    735

[547 rows x 4 columns]


In [93]:
# 3.9 Display the dataset showing East sales with units over 30 
east_sales_over_30 = df[(df['region'] == 'East') & (df['units'] > 30)]
print(east_sales_over_30)

           region                 type  units  sales
date                                                
2020-11-03   East  Children's Clothing   34.0    374
2020-06-12   East     Women's Clothing   35.0   1050
2020-05-11   East       Men's Clothing   35.0    700
2020-06-14   East       Men's Clothing   32.0    416
2020-10-28   East       Men's Clothing   34.0    986
...           ...                  ...    ...    ...
2020-02-24   East     Women's Clothing   32.0    693
2020-08-13   East       Men's Clothing   32.0    132
2020-10-04   East       Men's Clothing   35.0    350
2020-10-20   East       Men's Clothing   32.0    928
2020-02-11   East  Children's Clothing   35.0    735

[70 rows x 4 columns]


In [ ]:
# 3.10 A data corruption has been detected. Add 1 to all units. 
df['Units'] += 1

In [96]:
# 3.11 Another issue was detected, in West region add 2 to all units
df.loc[df['region'] == 'West', 'units'] += 2
